In [24]:
!wget "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

--2022-10-04 16:49:44--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.77.173.59, 2600:141b:e800:1483::317f, 2600:141b:e800:148c::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.77.173.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip.1’

kagglecatsanddogs_5 100%[===================>] 786.67M   219MB/s    in 3.6s    

2022-10-04 16:49:48 (217 MB/s) - ‘kagglecatsanddogs_5340.zip.1’ saved [824887076/824887076]



In [4]:
!unzip -q "kagglecatsanddogs_5340.zip"

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
from tqdm import tqdm

In [5]:
len(os.listdir("PetImages/Dog"))

12501

In [6]:
len(os.listdir("PetImages/Cat"))

12501

In [7]:
DataPath = pathlib.Path("PetImages")
all_paths = DataPath.glob("*/*.jpg")
all_paths 

<generator object Path.glob at 0x7fbe938a7e50>

In [8]:
all_paths = list(all_paths)
all_paths[:10]

[PosixPath('PetImages/Cat/2897.jpg'),
 PosixPath('PetImages/Cat/11252.jpg'),
 PosixPath('PetImages/Cat/2215.jpg'),
 PosixPath('PetImages/Cat/7325.jpg'),
 PosixPath('PetImages/Cat/7657.jpg'),
 PosixPath('PetImages/Cat/9304.jpg'),
 PosixPath('PetImages/Cat/4324.jpg'),
 PosixPath('PetImages/Cat/9308.jpg'),
 PosixPath('PetImages/Cat/3091.jpg'),
 PosixPath('PetImages/Cat/3214.jpg')]

In [9]:
all_paths = list(map(lambda x: str(x), all_paths))
all_paths[:10]

['PetImages/Cat/2897.jpg',
 'PetImages/Cat/11252.jpg',
 'PetImages/Cat/2215.jpg',
 'PetImages/Cat/7325.jpg',
 'PetImages/Cat/7657.jpg',
 'PetImages/Cat/9304.jpg',
 'PetImages/Cat/4324.jpg',
 'PetImages/Cat/9308.jpg',
 'PetImages/Cat/3091.jpg',
 'PetImages/Cat/3214.jpg']

In [10]:
from random import shuffle

shuffle(all_paths)
all_paths[:10]

['PetImages/Cat/9415.jpg',
 'PetImages/Dog/586.jpg',
 'PetImages/Dog/11381.jpg',
 'PetImages/Dog/8718.jpg',
 'PetImages/Cat/813.jpg',
 'PetImages/Cat/10038.jpg',
 'PetImages/Dog/2486.jpg',
 'PetImages/Cat/1597.jpg',
 'PetImages/Dog/10375.jpg',
 'PetImages/Dog/11477.jpg']

In [11]:
def TestImageQuality(all_paths):
  new_all_paths = []
  for path in tqdm(all_paths):
    try:
      image = tf.io.read_file(path)
      image = tf.io.decode_jpeg(image, channels = 3)
    except:
      continue
    new_all_paths.append(path)
  return new_all_paths

all_paths = TestImageQuality(all_paths)
all_paths[:10]

100%|██████████| 25000/25000 [00:21<00:00, 1172.10it/s]


['PetImages/Cat/9415.jpg',
 'PetImages/Dog/586.jpg',
 'PetImages/Dog/11381.jpg',
 'PetImages/Dog/8718.jpg',
 'PetImages/Cat/813.jpg',
 'PetImages/Cat/10038.jpg',
 'PetImages/Dog/2486.jpg',
 'PetImages/Cat/1597.jpg',
 'PetImages/Dog/10375.jpg',
 'PetImages/Dog/11477.jpg']

In [12]:
# Getting their respective labels
def get_label(image_path):
  return image_path.split("/")[-2]
all_labels = list(map(lambda x: get_label(x), all_paths))
all_labels[:10]

['Cat', 'Dog', 'Dog', 'Dog', 'Cat', 'Cat', 'Dog', 'Cat', 'Dog', 'Dog']

In [13]:
from sklearn.preprocessing import LabelEncoder

Le = LabelEncoder()
all_labels = Le.fit_transform(all_labels)

all_labels[:10]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1])

In [14]:
from sklearn.model_selection import train_test_split
Train_paths, Val_paths, Train_labels, Val_labels = train_test_split(all_paths, all_labels)

In [15]:
# Function used to decode jpeg into tensor
def load(image, label):
  image = tf.io.read_file(image)
  image = tf.io.decode_jpeg(image, channels = 3)
  return image, label

In [16]:
Train_paths[:10], Train_labels[:10]

(['PetImages/Dog/10747.jpg',
  'PetImages/Cat/799.jpg',
  'PetImages/Cat/4157.jpg',
  'PetImages/Dog/11803.jpg',
  'PetImages/Cat/5459.jpg',
  'PetImages/Dog/643.jpg',
  'PetImages/Cat/10083.jpg',
  'PetImages/Cat/9015.jpg',
  'PetImages/Cat/6715.jpg',
  'PetImages/Dog/10638.jpg'],
 array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1]))

In [17]:
Val_paths[:10], Val_labels[:10]

(['PetImages/Dog/11996.jpg',
  'PetImages/Dog/5745.jpg',
  'PetImages/Dog/1811.jpg',
  'PetImages/Dog/6770.jpg',
  'PetImages/Cat/8336.jpg',
  'PetImages/Cat/3119.jpg',
  'PetImages/Cat/2766.jpg',
  'PetImages/Cat/3814.jpg',
  'PetImages/Cat/730.jpg',
  'PetImages/Dog/7277.jpg'],
 array([1, 1, 1, 1, 0, 0, 0, 0, 0, 1]))

In [18]:
# Augmentation
IMG_SIZE = 224
BATCH_SIZE = 128

# Basic Transformation
resize = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, BATCH_SIZE)
])

# DATA Augmentation
data_augemntation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.3,-0.2))
])

In [27]:
# Function used to create a Tensorflow Data Object
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(paths , labels , train = True):
  image_paths = tf.convert_to_tensor(paths)
  labels = tf.convert_to_tensor(labels)

  image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
  label_dataset = tf.data.Dataset.from_tensor_slices(labels)
  
  dataset = tf.data.Dataset.zip((image_dataset , label_dataset))

  dataset = dataset.map(lambda image, label : load (image , label))
  dataset = dataset.map(lambda image, label: (resize(image), label), num_parallel_calls=2)
  dataset = dataset.shuffle(1000)
  dataset = dataset.batch(BATCH_SIZE)
  if train:
    dataset = dataset.map(lambda image, label: (data_augemntation(image), label), num_parallel_calls=2)
    dataset = dataset.repeat()
  return dataset

In [25]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
print(AUTOTUNE)

-1


In [28]:
%time train_dataset = get_dataset(Train_paths, Train_labels)

image, label = next(iter(train_dataset))
print(image.shape)
print(label.shape)

CPU times: user 161 ms, sys: 5.84 ms, total: 167 ms
Wall time: 167 ms
(128, 224, 128, 3)
(128,)


In [29]:
%time val_dataset = get_dataset(Val_paths, Val_labels, train=False)

image, label = next(iter(val_dataset))
print(image.shape)
print(label.shape)

CPU times: user 15.6 ms, sys: 1.97 ms, total: 17.6 ms
Wall time: 19.1 ms
(128, 224, 128, 3)
(128,)


In [31]:
# Model part
# CNN architecture inspered by GoogleNets 2014
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalMaxPool2D

model = Sequential()

# Block 1
model.add(Conv2D(input_shape=(224, 128,3), padding="same", filters=32, kernel_size=(7, 7)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block 2
model.add(Conv2D(filters=64, padding="valid", kernel_size=(5, 5)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block 3
model.add(Conv2D(filters=128, padding="valid", kernel_size=(5, 5)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


# Block 4
model.add(Conv2D(filters=256, kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(GlobalMaxPool2D())

model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(units=1))
model.add(Activation('sigmoid'))


In [32]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 128, 32)      4736      
                                                                 
 activation_6 (Activation)   (None, 224, 128, 32)      0         
                                                                 
 batch_normalization_4 (Batc  (None, 224, 128, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 64, 32)      0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 112, 64, 32)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 108, 60, 64)      

In [33]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics='accuracy'
)

In [34]:
# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=len(Train_paths)// BATCH_SIZE,
    epochs=10,
    validation_data=val_dataset,
    validation_steps = len(Val_paths)//BATCH_SIZE
)

Epoch 1/10
145/145 [==============================] - 1895s 13s/step - loss: 0.8691 - accuracy: 0.5695 - val_loss: 0.6511 - val_accuracy: 0.6204
Epoch 2/10
  6/145 [>.............................] - ETA: 28:57 - loss: 0.6369 - accuracy: 0.6533

KeyboardInterrupt: ignored